In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from api_key import g_key

In [2]:
vaca = pd.read_csv('/Users/kscomputer/Desktop/python_api_challenge/python-api-challenge/Weather/Output/weather.csv')
vaca

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speeds,Country,Date
0,0,tuktoyaktuk,69.45,-133.04,2.16,91,83,7.38,CA,1604145470
1,1,hasaki,35.73,140.83,46.40,93,20,1.12,JP,1604145470
2,2,port lavaca,28.61,-96.63,61.00,75,1,4.70,US,1604145340
3,3,saskylakh,71.92,114.08,-2.92,85,1,5.79,RU,1604145470
4,4,broken hill,-31.95,141.43,60.80,67,0,7.31,AU,1604145470
...,...,...,...,...,...,...,...,...,...,...
541,541,cidreira,-30.18,-50.21,67.01,62,89,10.16,BR,1604145511
542,542,zhezkazgan,47.80,67.71,48.58,44,0,8.41,KZ,1604145511
543,543,lahaina,20.88,-156.68,77.00,65,90,9.17,US,1604145511
544,544,kabarnet,0.49,35.74,73.40,49,40,11.41,KE,1604145511


In [3]:
lat_lng = vaca[['Lat','Lng']]

humidity = vaca['Humidity'].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(lat_lng, weights = humidity,
                                dissipating=False, max_intensity=100, 
                                 point_radius =4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#ran into issue in next cell getting lat and lng with na values why dropna included
ideal = vaca.loc[((vaca['Max Temp']<=80)&(vaca['Max Temp']>=70)&
                  (vaca['Wind Speeds']<=10)&
                  (vaca['Cloudiness'].astype(float)==0))].dropna(how='all')
ideal.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speeds,Country,Date
57,57,hermanus,-34.42,19.23,73.99,44,0,10.00,ZA,1604145357
104,104,esfahan,32.66,51.68,73.40,11,0,4.70,IR,1604145478
171,171,asadabad,34.78,48.12,75.20,11,0,4.70,IR,1604145484
174,174,khuzdar,27.74,66.64,74.30,17,0,5.03,PK,1604145484
238,238,dalbandin,28.89,64.41,75.78,14,0,3.00,PK,1604145489


In [6]:
hotel = ideal.loc[:,['City','Country','Lat','Lng']]
hotel['Hotel Name']=''
hotel

,City,Country,Lat,Lng,Hotel Name
57,hermanus,ZA,-34.42,19.23,
104,esfahan,IR,32.66,51.68,
171,asadabad,IR,34.78,48.12,
174,khuzdar,PK,27.74,66.64,
238,dalbandin,PK,28.89,64.41,
302,cap malheureux,MU,-19.98,57.61,
319,grand gaube,MU,-20.01,57.66,
375,vinukonda,IN,16.05,79.75,
456,tirano,IT,46.22,10.17,
463,nalut,LY,30.33,10.85,


In [11]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 5000
target_type = 'hotel'

# use iterrows to iterate through pandas dataframe
for index, row in hotel.iterrows():
    
    params = {'location':f"{row['Lat']},{row['Lng']}",
          'radius':target_radius,
            'type':target_type,
            'key':g_key
}

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results'][0]['name']
    
    hotel.loc[index, 'Hotel Name']=results
        


IndexError: list index out of range

In [ ]:
hotel.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
location = hotel[["Lat", "Lng"]]

In [ ]:
marker_layer = gmaps.marker_layer(location, info_box_content= hotel_info)

fig.add_layer(marker_layer)

fig

